<a href="https://colab.research.google.com/github/ThaoPham96/DeepARV/blob/main/DeepARV_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Set up


In [2]:
pip install tensorboard

In [3]:
import tensorflow as tf
import keras
from numpy import argmax
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
import math
import tensorflow.experimental.numpy as tnp
import timeit
import datetime, os
import pandas as pd
import numpy as np

In [4]:
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import classification_report
import keras
from numpy import argmax
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Activation, Dropout
import math
import tensorflow.experimental.numpy as tnp
import timeit
import datetime, os

In [5]:
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
!pip install tensorflow_addons
import tensorflow_addons as tfa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.1/612.1 kB 10.2 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


#Load data

In [7]:
# Loading training dataset
  ## n = 23,318
  ## X represents drug pair between 34 ARV and 695 comedication
  ## y represents DDI classification
X_train = np.load('/content/drive/MyDrive/liverpool_hiv/Cleaning_data/numpy_array/X_train_concat.npy')
y_train = np.load('/content/drive/MyDrive/liverpool_hiv/Cleaning_data/numpy_array/Y_train_full.npy')

In [8]:
# Loading testing dataset (8 untrained ARV and 695 comedication)
  ## n = 5824 drug pairs
  ## X represents drug pair between ARV and comedication
  ## y represents DDI classification
X_test = np.load('/content/drive/MyDrive/liverpool_hiv/Cleaning_data/numpy_array/X_test_concat.npy')
y_test = np.load('/content/drive/MyDrive/liverpool_hiv/Cleaning_data/numpy_array/Y_test_full.npy')

In [9]:
# Distribution of DDI class in the training set
unique, counts = np.unique(y_train, return_counts=True)
print(np.asarray((unique, counts)).T)
# DDI risks:
  ## 0 = Green: No DDI expected
  ## 1 = Yellow: Potential interaction of weak clinical relevance
  ## 2 = Amber: Potential clinically relevant interaction
  ## 3 = Red: These drugs should not be co-administered

[[    0 16745]
 [    1  1782]
 [    2  3728]
 [    3  1063]]


In [10]:
# Distribution of DDI class in the testing set
unique, counts = np.unique(y_test, return_counts=True)
print(np.asarray((unique, counts)).T)
# DDI risks:
  ## 0 = Green: No DDI expected
  ## 1 = Yellow: Potential interaction of weak clinical relevance
  ## 2 = Amber: Potential clinically relevant interaction
  ## 3 = Red: These drugs should not be co-administered

[[   0 3740]
 [   1  416]
 [   2 1234]
 [   3  434]]


#Function

###Confusion matrix

In [11]:
#This function plots confusion matrix

def plot_confusion_matrix(cf_matrix,
                          x_class_labels, y_class_labels,
                          axis_x_title,axis_y_title):

    cf_matrix = np.array(cf_matrix)
    group_counts = [value for value in cf_matrix.flatten()]
    labels = [f"{v1}"
              for v1 in
              group_counts]
    labels= np.asarray(labels).reshape(len(cf_matrix),
                                       len(cf_matrix))
    sns.set(font_scale=1)
    from matplotlib import pyplot
    pyplot.figure(
                  figsize=(5, 2),
                  dpi=150
                  ) # width and height in inches
    res = sns.heatmap(
                      cf_matrix, annot=labels,
                      fmt='', annot_kws={'size': 11.5},
                      cmap='Blues',
                      xticklabels = x_class_labels,
                      yticklabels = y_class_labels
                      )
    res.set_xticklabels(
                        res.get_xmajorticklabels(),
                        fontsize = 10.5
                        )
    res.set_yticklabels(
                        res.get_ymajorticklabels(),
                        fontsize = 10.5
                        )
    plt.xlabel(axis_x_title, size=11,fontweight='bold')
    plt.ylabel(axis_y_title, size=11,fontweight='bold')

In [12]:
#This function plots heatmap
#for a given confusion matrix
        #require: confusion matrix,
        #...classes for x and y axis,
        #...and title label for x and y axis

def plot_normalised_cf(cf_matrix,
                          x_class_labels, y_class_labels,
                          axis_x_title,axis_y_title):

    #cf_matrix = np.array(cf_matrix)
    cf_matrix_cal = np.array(cf_matrix)/np.sum(cf_matrix,axis=1)[:,np.newaxis]
    cf_matrix_normalised = np.around(cf_matrix_cal,2)
    group_counts = [value for value in cf_matrix_normalised.flatten()]
    labels = [f"{v1}"
              for v1 in
              group_counts]
    labels= np.asarray(labels).reshape(len(cf_matrix_normalised),
                                       len(cf_matrix_normalised))
    sns.set(font_scale=1)
    from matplotlib import pyplot
    pyplot.figure(
                  figsize=(5, 2),
                  dpi=150
                  ) # width and height in inches
    res = sns.heatmap(
                      cf_matrix_normalised, annot=labels,
                      fmt='', annot_kws={'size': 11.5},
                      cmap='Purples',
                      xticklabels = x_class_labels,
                      yticklabels = y_class_labels
                      )
    res.set_xticklabels(
                        res.get_xmajorticklabels(),
                        fontsize = 10.5
                        )
    res.set_yticklabels(
                        res.get_ymajorticklabels(),
                        fontsize = 10.5
                        )
    plt.xlabel(axis_x_title, size=11,fontweight='bold')
    plt.ylabel(axis_y_title, size=11,fontweight='bold')

###Metric for each class

In [13]:
#This function calculates true negative (tn),
    #false positive (fp), false negative (fn),
    #and true positive (tp) for each DDI class
    #returns: accuracy, precision, sensitivity,
    #specificity, f1 score and balanced accuracy

def metrics_func(matrix):
  tn = matrix[0][0]
  fp = matrix[1][0]
  fn = matrix[0][1]
  tp = matrix[1][1]

  accuracy = (tp+tn)/(tp+tn+fp+fn)
  precision = tp/(tp+fp)
  specificity = tn/(tn+fp)
  sensitivity = tp/(tp+fn)
  f1_score = (2*((precision*sensitivity) /
                   (precision+sensitivity)))
  BAcc = (sensitivity + specificity) / 2
  return accuracy, precision, sensitivity, specificity, f1_score, BAcc

## 5-fold cross validation

In [14]:
#create feedforward neural network model
def build_model(feature):
  model = Sequential()
  model.add(Dense(1024, activation='relu', input_shape=(feature,))),
  model.add(Dropout(0.2)),
  model.add(Dense(512, activation='relu')),
  model.add(Dropout(0.2)),
  model.add(Dense(256, activation='relu')),
  model.add(Dropout(0.2)),
  model.add(Dense(128, activation='relu')),
  model.add(Dropout(0.2)),
  model.add(Dense(4, activation='softmax'))
  tf.keras.optimizers.Adam(learning_rate=0.001,
                          beta_1=0.9, beta_2=0.999,
                          epsilon=1e-07,
                          amsgrad=False,
                          name='Adam')
  model.compile(optimizer='Adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

In [16]:
#This function perform stratified 5-fold split for each DDI class
## given the dataset
def classSplit(pd_df):
    class_num = 4
    class_dict = {}
    for i in range(class_num):
        class_dict[i] = pd_df[pd_df.Label == i].reset_index(drop=True)
    class0_split = [class_dict[0][j::5] for j in range(5)]
    class1_split = [class_dict[1][j::5] for j in range(5)]
    class2_split = [class_dict[2][j::5] for j in range(5)]
    class3_split = [class_dict[3][j::5] for j in range(5)]

    fold = {}
    frame ={}
    for k in range(5):
        frame[k] = [class0_split[k],class1_split[k],
                    class2_split[k], class3_split[k]]
        fold[k] = pd.concat(frame[k],ignore_index=True)
    return fold

In [17]:
#Downsampling Green (0) to same size as Amber (2)
def downsampleMajority(df, n_models,n):
  '''
  Splits majority samples into n_models splits
  df - df containing majoirt
  n_models - number of models/data splits
  n - number of samples per data split
  '''
  # Dictionary to store majority splits
  majority_splits = {}

  for i in range(n_models):
    if i < (n_models-1):

      start = i*n
      end = (i+1)*n
      majority_splits[i] = df.iloc[start:end,:]
    else:
      majority_splits[i] = pd.concat([df.iloc[end:,:],df.sample(n=1604)])

  return majority_splits

In [18]:
#Combine majority and minority
def trainCVSplit(train_cv):
    minority = train_cv[train_cv.Label != 0]
    majority = train_cv[train_cv.Label == 0].reset_index(drop=True
                       ).sample(frac=1, random_state=42).reset_index(drop=True)

    #use downsampleMajority function to downsample majoriy
    majority_dict = downsampleMajority(majority,5,3000)
    train_split_dict = {} #dict to hold final training df
    n_model = 5
    feature_dict = {}
    label_dict = {}

    for i in range(5):
        train_split_dict[i] = pd.concat([majority_dict[i],minority]
                                        ).reset_index(drop=True
                                        ).sample(frac=1,random_state=42
                                        ).reset_index(drop=True)
        feature_dict[i] = train_split_dict[i].drop(columns='Label')
        label_dict[i] = train_split_dict[i].Label

    return feature_dict, label_dict

In [19]:
from sklearn.utils import class_weight

In [20]:
# This function builds model and perform cross-validation
def singleCV(train,test, model_dict):
    # undersample
    feature, label = trainCVSplit(train)

    # Build models and train
    model_dict = {}
    for i in range(5):
        class_weights = class_weight.compute_class_weight(
                                                  class_weight='balanced',
                                                  classes = np.unique(label[i]),
                                                  y = label[i])
        class_weights_model = dict(zip(np.unique(label[i]), class_weights))
        print(class_weights_model)
        print(class_weights)
        model_dict[i] = build_model(1374)
        model_dict[i].fit(feature[i],label[i],epochs=30,
                          validation_split=0.1,
                          batch_size = 128, class_weight=class_weights_model,
                          callbacks=[tensorboard_callback])

    # Predict for all models
    all_results = {}
    temp_results = {}
    for i in range(5):
        #make prediction
        all_results[i] = model_dict[i].predict(test.iloc[:,:-1])

        #store each final class prediction per model in temp dict
        class_pred = []
        ##loop through each prediction and get the class prediction
        for j in all_results[i]:
            prob_avg = np.average(j)
            class_pred.append(np.argmax(prob_avg))
        temp_results[i] = class_pred

    # Vote max
    df_vote = pd.DataFrame(data=temp_results)
    df_vote['final_vote'] = df_vote.mode(axis=1).astype(int)

    y_p_ohe = tf.keras.utils.to_categorical(df_vote.final_vote)
    y_t_ohe = tf.keras.utils.to_categorical(test.iloc[:,-1])
    metric = tfa.metrics.MultiLabelConfusionMatrix(num_classes=4)
    metric.update_state(y_p_ohe, y_t_ohe)
    result = metric.result()
    result = result.numpy()
    class_0 = metrics_func(result[0])
    class_1 = metrics_func(result[1])
    class_2 = metrics_func(result[2])
    class_3 = metrics_func(result[3])

    # Macro metrics
    #accuracy, precision,  sensitivity, specificity, f1_score, BAcc
    macro = np.mean((class_0, class_1, class_2, class_3), axis=0)
    #print('\n MACOR', macro)
    macros = macro.reshape(1,-1)
    each_class_results = np.vstack((class_0,class_1,class_2,class_3,macros))

    return macros, each_class_results



##Execute

In [21]:
!rm -rf logs/
from tensorflow.keras.callbacks import TensorBoard
%load_ext tensorboard
log_folder = 'logs'

In [22]:
%reload_ext tensorboard
logdir = os.path.join("logs/exp2", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1,
                                                      write_graph=True,
                                                      write_images=True,
                                                      update_freq='epoch',
                                                      profile_batch=2,
                                                      embeddings_freq=1)

In [23]:
# 1. split the class
# create df from x and y train
train_df = pd.DataFrame(data=X_train)
train_df['Label'] = y_train
train_df

# func. to split class
train_5fold = classSplit(train_df)

# 2. iterate 1 fold for testing/evaluation and 4 fold for training

#Build 5 models
model_dict = {}
for i in range(5):
    model_dict[i] = build_model(1374)


#Evaluation
results = np.zeros((5,6))
general_results = []
for f in range(5):
    #reset folds
    folds = [0,1,2,3,4]
    test_idx = folds.pop(f) # take 0...4 from list
    test_ds = train_5fold[test_idx]
    train_ds = pd.concat([train_5fold[folds[0]], train_5fold[folds[1]],
                        train_5fold[folds[2]], train_5fold[folds[3]]])

    res, all_res = singleCV(train_ds,test_ds, model_dict)

    results[f] = res
    general_results.append(all_res)
    #%reload_ext tensorboard
    #logdir = os.path.join("logs/exp2", datetime.datetime.now().strftime)
    #tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir,
                                                         #histogram_freq=1,
                                                         #write_graph=True,
                                                         #write_images=True,
                                                         #update_freq='epoch',
                                                         #profile_batch=2,
                                                         #embeddings_freq=1)

{0: 0.6880833333333334, 1: 1.4485964912280702, 2: 0.6922367538564722, 3: 2.428529411764706}
[0.68808333 1.44859649 0.69223675 2.42852941]
Epoch 1/30
59/59 [==============================] - 17s 66ms/step - loss: 1.3856 - accuracy: 0.2589 - val_loss: 1.2986 - val_accuracy: 0.3692
Epoch 2/30
59/59 [==============================] - 3s 44ms/step - loss: 1.2915 - accuracy: 0.4003 - val_loss: 1.1815 - val_accuracy: 0.4467
Epoch 3/30
59/59 [==============================] - 2s 43ms/step - loss: 1.1826 - accuracy: 0.4450 - val_loss: 1.0925 - val_accuracy: 0.4988
Epoch 4/30
59/59 [==============================] - 3s 54ms/step - loss: 1.1036 - accuracy: 0.4972 - val_loss: 1.9492 - val_accuracy: 0.2494
Epoch 5/30
59/59 [==============================] - 2s 32ms/step - loss: 1.1374 - accuracy: 0.4784 - val_loss: 1.1175 - val_accuracy: 0.4431
Epoch 6/30
59/59 [==============================] - 1s 25ms/step - loss: 1.0360 - accuracy: 0.5204 - val_loss: 1.0254 - val_accuracy: 0.5484
Epoch 7/30
59/5